In [1]:
%pwd

'c:\\Users\\PC\\Documents\\mlops\\research'

In [4]:
os.chdir("../")

In [2]:
import os


In [5]:
%pwd

'c:\\Users\\PC\\Documents\\mlops'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_urls:str
    local_data:Path
    unzip_dir:Path

In [25]:
from cnn.constants import *
from cnn.utils.common import read_yaml,make_dir

class ConfigurationManager:
    def __init__(
            self,
            params_file= PARAMS_FILE_PATH,
            config_file= CONFIG_FILE_PATH):
            
        self.config=read_yaml(config_file)
        self.params=read_yaml(params_file)

        make_dir([self.config.artifacts_root])

    

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion

        make_dir([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_urls=config.source_urls,
            local_data=config.local_data,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config
        

    


NameError: name 'PARAMS_FILE_PATH' is not defined

In [ ]:
import os
import zipfile
import gdown
from cnn import logger
from cnn.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config



    def download_file(self)->str:

        try:
            dataset_url=self.config.source_urls
            zip_download_dir=self.config.local_data
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"downloading data from {dataset_url} into directory {zip_download_dir}")

            file_id=dataset_url.split("/")[-2]
            prefix="https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f"downloaded data from {dataset_url} into directory {zip_download_dir}")

        except Exception as e:
            raise e
        

    def extract_zip(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data,'r') as zip:
            zip.extractall(unzip_path)




In [ ]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip()

except Exception as e:
    raise e